# Convert all sports dfs into one

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, time
today = pd.Timestamp.today().date()

import pytz

## Read

In [2]:
# Get the current year
current_year = datetime.now().year
next_year = current_year + 1

# Define a custom date parser for the corresponding formats
fb_date_parser = lambda x: pd.to_datetime(x, format="%d/%m/%y").date()
nba_nhl_date_parser = lambda x: pd.to_datetime(x, format="%a, %b %d, %Y").date()
nfl_date_parser = lambda x: pd.to_datetime(f"{x}, {next_year}" if "Jan" in x else f"{x}, {current_year}", format="%a, %b %d, %Y").date() if pd.notna(x) else np.nan


# Function for home/away cols for nba/nhl/nfl
def assign_teams(df, home_away_col, user_team_col, opponent_col):
    df['Home Team'] = np.where(df[home_away_col] == 'vs.', df[user_team_col], df[opponent_col])
    df['Away Team'] = np.where(df[home_away_col] == '@', df[user_team_col], df[opponent_col])
    return df

In [3]:
# Read in files with their dates
## football/soccer
df_fb = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_fb_master_2024_10_28.csv",
    parse_dates=['Date'], 
    date_parser=fb_date_parser
)
### Create/convert cols as necessary
df_fb['Date'] = pd.to_datetime(df_fb['Date']).dt.date
df_fb = df_fb[df_fb['Date'] >= today]
df_fb['Time'] = df_fb['Time'].astype(str).str.replace(r'\s+', '', regex=True)
df_fb['Time'] = df_fb['Time'].replace('-', pd.NA)
df_fb['Time'] = pd.to_datetime(df_fb['Time'], format='%H:%M').dt.time


## nba
df_nba = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nba_master_2024_10_29.csv",
    parse_dates=['Date'], 
    date_parser=nba_nhl_date_parser
)
### Create/convert cols as necessary
df_nba['Date'] = pd.to_datetime(df_nba['Date']).dt.date
df_nba = assign_teams(df_nba, 'Home/Away', 'user_team', 'Opponent')
df_nba['Start (ET)'] = pd.to_datetime(df_nba['Start (ET)'], infer_datetime_format=True).dt.time
df_nba = df_nba.rename(columns={'Start (ET)': 'Time'})
df_nba['Time'] = pd.to_datetime(df_nba['Time'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nba['Time'] = df_nba['Time'].dt.time


## nhl
df_nhl = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nhl_master_2024_10_30.csv",
    parse_dates=['Date'], 
    date_parser=nba_nhl_date_parser
)
### Create/convert cols as necessary
df_nhl['Date'] = pd.to_datetime(df_nhl['Date']).dt.date
df_nhl = assign_teams(df_nhl, 'Home/Away', 'user_team', 'Opponent')
df_nhl['Time'] = pd.to_datetime(df_nhl['Time'], infer_datetime_format=True).dt.time
df_nhl['Time'] = pd.to_datetime(df_nhl['Time'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nhl['Time'] = df_nhl['Time'].dt.time


## nfl
df_nfl = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nfl_master_2024_11_01.csv",
    parse_dates=['DATE'], 
    date_parser=nfl_date_parser
)
df_nfl['DATE'] = pd.to_datetime(df_nfl['DATE']).dt.date
df_nfl = assign_teams(df_nfl, 'Home/Away', 'user_team', 'OPPONENT')
df_nfl['TIME'] = pd.to_datetime(df_nfl['TIME'], infer_datetime_format=True).dt.time
df_nfl['TIME'] = pd.to_datetime(df_nfl['TIME'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nfl['TIME'] = df_nfl['TIME'].dt.time

## Combine fb, nba, nhl, nfl

In [16]:
# First, rename and reorder columns in df_fb
df_fb_fin = df_fb.rename(columns={'Team': 'user_team', 'League': 'game_type'})
df_fb_fin = df_fb_fin[['user_team', 'Date', 'game_type', 'Home Team', 'Away Team', 'Time']].copy()
df_fb_fin['Sport'] = 'Soccer'

# Select and reorder columns in df_nba and df_nhl
fin_cols_list = ['user_team', 'Date', 'game_type', 'Home Team', 'Away Team', 'Time']
df_nba_fin = df_nba[fin_cols_list].copy()
df_nba_fin['Sport'] = 'NBA'

df_nhl_fin = df_nhl[fin_cols_list].copy()
df_nhl_fin['Sport'] = 'NHL'

# Select and reorder columns in df_nfl
df_nfl_fin = df_nfl[['user_team','DATE','game_type','Home Team','Away Team', 'TIME']].copy()
df_nfl_fin.columns = fin_cols_list
df_nfl_fin['Sport'] = 'NFL'

In [17]:
# Stack all dataframes and Sort the combined dataframe by Date and then by Time, both in ascending order
df_fin_all = (
    pd.concat([df_fb_fin, df_nba_fin, df_nhl_fin, df_nfl_fin], ignore_index=True)
    .sort_values(by=['Date', 'Time', 'Home Team'], ascending=[True, True, True])
    .reset_index(drop=True)
)

In [19]:
df_fin_future = df_fin_all[df_fin_all['Date'] >= today]
df_fin_future

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
474,Dallas Stars,2024-11-01,regular season,Dallas Stars,Florida Panthers,11:00:00,NHL
475,Florida Panthers,2024-11-01,regular season,Dallas Stars,Florida Panthers,11:00:00,NHL
476,Schalke 04,2024-11-01,2.B,Ulm,Schalke 04,11:30:00,Soccer
477,Monaco,2024-11-01,LI1,Monaco,Angers,12:00:00,Soccer
478,Angers,2024-11-01,LI1,Monaco,Angers,12:00:00,Soccer
...,...,...,...,...,...,...,...
7645,Toulouse,2025-05-17,LI1,Saint-Etienne,Toulouse,NaT,Soccer
7646,Strasbourg,2025-05-17,LI1,Strasbourg,Le Havre,NaT,Soccer
7647,FC Cologne,2025-05-18,2.B,FC Cologne,Kaiserslautern,07:30:00,Soccer
7648,Hertha Berlin,2025-05-18,2.B,Hertha Berlin,Hannover 96,07:30:00,Soccer


In [20]:
df_fin_future[df_fin_future['Sport'].isin(['NBA','NHL'])]

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
474,Dallas Stars,2024-11-01,regular season,Dallas Stars,Florida Panthers,11:00:00,NHL
475,Florida Panthers,2024-11-01,regular season,Dallas Stars,Florida Panthers,11:00:00,NHL
487,Buffalo Sabres,2024-11-01,regular season,Buffalo Sabres,New York Islanders,17:00:00,NHL
488,New York Islanders,2024-11-01,regular season,Buffalo Sabres,New York Islanders,17:00:00,NHL
489,Boston Celtics,2024-11-01,regular season,Charlotte Hornets,Boston Celtics,17:00:00,NBA
...,...,...,...,...,...,...,...
7461,Ottawa Senators,2025-04-17,regular season,Ottawa Senators,Carolina Hurricanes,17:00:00,NHL
7462,Pittsburgh Penguins,2025-04-17,regular season,Pittsburgh Penguins,Washington Capitals,17:00:00,NHL
7463,Washington Capitals,2025-04-17,regular season,Pittsburgh Penguins,Washington Capitals,17:00:00,NHL
7464,Detroit Red Wings,2025-04-17,regular season,Toronto Maple Leafs,Detroit Red Wings,17:00:00,NHL


In [21]:
df_fin_future[df_fin_future['Sport'] == 'NFL']

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
632,Dallas Cowboys,2024-11-03,regular season,Atlanta,Dallas Cowboys,11:00:00,NFL
633,Atlanta Falcons,2024-11-03,regular season,Atlanta Falcons,Dallas,11:00:00,NFL
634,Denver Broncos,2024-11-03,regular season,Baltimore,Denver Broncos,11:00:00,NFL
635,Baltimore Ravens,2024-11-03,regular season,Baltimore Ravens,Denver,11:00:00,NFL
636,Miami Dolphins,2024-11-03,regular season,Buffalo,Miami Dolphins,11:00:00,NFL
...,...,...,...,...,...,...,...
3147,Philadelphia Eagles,2024-12-29,regular season,Philadelphia Eagles,Dallas,14:25:00,NFL
3168,Miami Dolphins,2024-12-29,regular season,Cleveland,Miami Dolphins,18:20:00,NFL
3169,Cleveland Browns,2024-12-29,regular season,Cleveland Browns,Miami,18:20:00,NFL
3194,Detroit Lions,2024-12-30,regular season,San Francisco,Detroit Lions,18:15:00,NFL
